In [5]:
import cv2 as cv
import numpy as np

RED= (0,0,255)
BLUE= (255,0,0)

face_cascade= cv.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade= cv.CascadeClassifier('haarcascade_eye.xml')

blob_detector_params = cv.SimpleBlobDetector_Params()
blob_detector_params.filterByArea = True
blob_detector_params.filterByCircularity = True
blob_detector_params.filterByConvexity = True
blob_detector_params.maxArea = 1500
blob_detector = cv.SimpleBlobDetector_create(blob_detector_params)

#pic= cv.imread('face.jpg', 1)
#frame= pic

#Function to return largest detected face from a frame
def detect_face(frame, classifier):
    gray_frame= cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    faces= classifier.detectMultiScale(gray_frame, 1.3, 5)
    
    if len(faces) > 1:
        biggest= (0, 0, 0, 0)
        for face in faces:
            if(face[2]*face[3] > biggest[2]*biggest[3]):
                biggest= face
    elif len(faces) == 1:
        biggest = faces[0]
    else:
        return None
        
    x, y, w, h= biggest
    return frame[y:y+h, x:x+w]

#Function to detect left and right eye from a frame
def detect_eyes(face, classifier):
    gray_frame = cv.cvtColor(face, cv.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray_frame, 1.3, 5) # detect eyes
    #print(len(eyes), end= ' ')
    
    width = np.size(gray_frame, 1) # get face frame width
    height = np.size(gray_frame, 0) # get face frame height
    
    left_eye = None
    right_eye = None
    
    for (x, y, w, h) in eyes:
        if y > height / 2:
            continue
        
        eyecenter = x + w / 2  # get the eye center
        if eyecenter < width * 0.5:
            left_eye = face[y:y + h, x:x + w]
            #print("Right eye found")
        else:
            right_eye = face[y:y + h, x:x + w]
        
    return left_eye, right_eye

def cut_eyebrows(eye):
    
    height, _ = eye.shape[:2]
    eyebrow_height= height // 4
    return eye[eyebrow_height:height, :]


def detect_iris(eye, threshold):
    
    eye= cv.cvtColor(eye, cv.COLOR_BGR2GRAY)
    #cv.imshow('Original', eye)
    eye= cv.bilateralFilter(eye, 10, 10, 20)
    #cv.imshow('Bilateral', eye)
    _, eye= cv.threshold(eye, threshold, 255, cv.THRESH_BINARY)
    #cv.imshow('Threshed', eye)
    eye = cv.medianBlur(eye, 5) #3
    #cv.imshow('Blurred1', eye)
    eye = cv.erode(eye, None, iterations=1) #1
    #cv.imshow('Eroded', eye)
    eye = cv.dilate(eye, None, iterations= 2) #2
    #cv.imshow('Dilated', eye)
    eye = cv.medianBlur(eye, 5) #3
    #cv.imshow('Blurred2', eye)
    keypoints = blob_detector.detect(eye)  
    
    return keypoints, eye  
    
            

cap= cv.VideoCapture(0)

#480p
cap.set(3, 640)
cap.set(4, 480)

#720p
#cap.set(3, 1280)
#cap.set(4, 720)

def nothing(x):
    pass

cv.namedWindow('Frame')
cv.createTrackbar('threshold', 'Frame', 50, 255, nothing)
#cv.createTrackbar('erode_iter', 'Face', 0, 4, nothing)
#cv.createTrackbar('dilate_iter', 'Face', 0, 4, nothing)
while(True):
    
    ret, frame= cap.read()
    cv.imshow('Frame', frame)

    face= detect_face(frame, face_cascade)
    
    if face is not None:
        left_eye, right_eye= detect_eyes(face, eye_cascade)
        
        if left_eye is not None:
            left_eye= cut_eyebrows(left_eye) 
            threshold= cv.getTrackbarPos('threshold', 'Frame')                        
            keypoints, binary_left= detect_iris(left_eye, threshold)
            left_eye= cv.drawKeypoints(left_eye, keypoints, left_eye, RED, cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
            cv.imshow("Right Eye", binary_left)
            
        if right_eye is not None:
            right_eye= cut_eyebrows(right_eye) 
            threshold= cv.getTrackbarPos('threshold', 'Frame')                        
            keypoints, binary_right= detect_iris(right_eye, threshold)
            right_eye= cv.drawKeypoints(right_eye, keypoints, right_eye, RED, cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
            cv.imshow("Left Eye", binary_right)
        
        cv.imshow('Face', face)                 
        
        
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv.destroyAllWindows()